Combine u and v wind files

In [84]:
# Libraries
%load_ext autoreload
%autoreload


import numpy as np
import iris
import iris.coord_categorisation
from iris.util import unify_time_units
from iris.experimental.equalise_cubes import equalise_attributes
# import pandas as pd

import glob
import sys
sys.path.append('../')
from libs.plot_maps import *


import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy.ma as ma
%matplotlib inline
import cartopy.crs as ccrs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path     = '../data/aus_wind/'

# Extracts all the files in that folder
u_files = glob.glob(path + 'uwnd*.nc') # use 'recursive = True' to include subdirectories
u_files = sorted(u_files) # sort alphabetically

v_files = glob.glob(path + 'vwnd*.nc') 
print(u_files[0] + v_files[0])

../data/aus_wind/uwnd.10m.gauss.2001.nc../data/aus_wind/vwnd.10m.gauss.2001.nc


Concatenate all u files together and all v files together

In [64]:
# Load u files
uList = []
for i in u_files[0:3]:
    df = iris.load_cube(i)
#     print(df.coord('time').units)
#     print(df.coord('time').points)
    uList.append(df)
    print(uList)

# print(uList)
# u_cube_list = iris.cube.CubeList(uList)
# u = u_cube_list.concatenate_cube()

# # Load v files
# vList = []
# for i in v_files[0:2]:
#     print(i)
#     df = iris.load_cube(i)
#     vList.append(df)

# v_cube_list = iris.cube.CubeList(vList)
# v = v_cube_list.concatenate_cube()

[<iris 'Cube' of 4xDaily u-wind at 10 m / (m/s) (time: 1460; latitude: 94; longitude: 192)>]
[<iris 'Cube' of 4xDaily u-wind at 10 m / (m/s) (time: 1460; latitude: 94; longitude: 192)>, <iris 'Cube' of 4xDaily u-wind at 10 m / (m/s) (time: 1460; latitude: 94; longitude: 192)>]
[<iris 'Cube' of 4xDaily u-wind at 10 m / (m/s) (time: 1460; latitude: 94; longitude: 192)>, <iris 'Cube' of 4xDaily u-wind at 10 m / (m/s) (time: 1460; latitude: 94; longitude: 192)>, <iris 'Cube' of 4xDaily u-wind at 10 m / (m/s) (time: 1460; latitude: 94; longitude: 192)>]


In [85]:
u_cube_list = iris.cube.CubeList(uList)

unify_time_units(u_cube_list)
equalise_attributes(u_cube_list)
u = u_cube_list.concatenate_cube()



# print(u)
# print(u[0].coord('time').units, u[0].attributes)
# print(u[1].coord('time').units, u[1].attributes)
# print(u[2].coord('time').units, u[2].attributes)

ConcatenateError: failed to concatenate into a single cube.
  Dimension coordinates metadata differ: time != time

In [25]:
print(u)

0: 4xDaily u-wind at 10 m / (m/s)      (time: 1460; latitude: 94; longitude: 192)
1: 4xDaily u-wind at 10 m / (m/s)      (time: 1460; latitude: 94; longitude: 192) 0: 4xDaily v-wind at 10 m / (m/s)      (time: 1460; latitude: 94; longitude: 192)
1: 4xDaily v-wind at 10 m / (m/s)      (time: 1460; latitude: 94; longitude: 192)


### Calculate wind speed

For a single year, calculate `s` which is wind speed, given by:

$s = \sqrt(u^2 + v^2)$

In [26]:
# u = iris.load_cube(u_files[0])
# v = iris.load_cube(v_files[0])

# Have to load in data to do this (may take a while)
v_dat = v.data
u_dat = u.data
s_dat = np.sqrt((v_dat * v_dat) + (u_dat * u_dat))

# Put data in a cube, bit cheating but it works
s = v.copy()
s.data = s_dat

AttributeError: 'CubeList' object has no attribute 'data'

Take the 4 measurements per day and find the maximum wind speed. Add day time coordinate

In [4]:
# add_month, add_day_of_year, year

iris.coord_categorisation.add_month(s, 'time', name='month')
iris.coord_categorisation.add_day_of_year(s, 'time', name='day_of_year')
iris.coord_categorisation.add_year(s, 'time', name='year')

In [10]:
test = s.aggregated_by(['day_of_year'], iris.analysis.MAX)

for y in year:
    for d in day:
        if s.coord.day == d:
            s_daily_max = s.collapsed('day', iris.analysis.MAX)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 24

In [8]:
print(test, s)

4xDaily v-wind at 10 m / (m/s)      (time: 365; latitude: 94; longitude: 192)
     Dimension coordinates:
          time                           x              -              -
          latitude                       -              x              -
          longitude                      -              -              x
     Auxiliary coordinates:
          day_of_year                    x              -              -
          month                          x              -              -
     Attributes:
          Conventions: COARDS
          GRIB_id: 34
          GRIB_name: V GRD
          References: http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.html
          actual_range: [-30.199997  28.699997]
          dataset: NCEP Reanalysis
          dataset_title: NCEP-NCAR Reanalysis 1
          description: Data is from NMC initialized reanalysis
(4x/day).  It consists of T62 variables...
          history: created 00/12/18 by Hoop (netCDF2.3)
Converted to chunked, 